<a href="https://colab.research.google.com/github/zombienok/short_quest_generator/blob/main/quest_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zombienok/short_quest_generator.git
!cd short_quest_generator
!pip install -r /content/short_quest_generator/requirements.txt
!curl -fsSL https://ollama.com/install.sh | sh

Cloning into 'short_quest_generator'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 39 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 29.90 KiB | 7.47 MiB/s, done.
Resolving deltas: 100% (6/6), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.4 MB/s eta 0:00:00
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [22]:
import subprocess
import time
import ollama


# Запускаем сервер Ollama в фоне (если ещё не запущен)
proc = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ждём, пока сервер запустится (обычно 3-5 секунд)
time.sleep(3)

# Проверяем, работает ли сервер
try:
    ollama.pull("owl/t-lite:latest")
    print(ollama.list())  # Должен вывести список моделей (пустой, если ничего не загружено)
except Exception as e:
    print("Ошибка:", e)

models=[Model(model='owl/t-lite:latest', modified_at=datetime.datetime(2025, 8, 4, 0, 46, 25, 992189, tzinfo=TzInfo(UTC)), digest='52bcfdc27e1b605bb7905cfbfaca2cf5541e85f86c1e1138f0e00d11336242b1', size=4661229593, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_0'))]


In [ ]:
prompt_choices = """
Опиши варианты, последствия, описания сцен детальнее, сохраняя их количество.
Дополни варианты для сцен строго так:
branchA_scene1:
Смешной: Действие -> {ПОСЛЕДСТВИЕ} + branchA_scene2
Провальный: Действие -> failure_why
Логичный: Действие -> {ПОСЛЕДСТВИЕ} + branchA_scene3
Никаких символов *, -, "". Только текст по шаблону.
Ожидаемый формат (варианты могут быть другими):
-branchA_scene1:
  Смешной: "Станцевать тектоник" → {"Бармен присоединяется, теряет бдительность"} + branchA_scene2
  Провальный: "Выпить неизвестную жидкость" → "Отравление, game over"
  Логичный: "Подкупить охранника" → "branchA_scene3"
"""



In [26]:
import ollama
import json


MODEL_TO_USE = "owl/t-lite:latest"


messages_strict = [{
    "role": "system",
    "content": "Ты генератор структуры для квеста. Отвечаешь ТОЛЬКО в указанном формате. Никаких комментариев, пояснений, символов *, -, >."
}]
# вводные данные
COUNT_OF_FIRST_CHOICES = 2
CHOICES_AVG = 2
DEPTH = 3
COUNT_OF_SCENES = COUNT_OF_FIRST_CHOICES *(CHOICES_AVG * DEPTH) + 2

prompt_structure = f"""
Сгенерируй структуру квеста по следующим правилам:
1. Формат строго:
   <номер>. <scene_id>: <Выбор1> -> <target1>[, <Выбор2> -> <target2>]
2. Условия:
   - Минимум 8 уникальных сцен (не считая финалов/провалов)
   - В стартовой сцене {COUNT_OF_FIRST_CHOICES} выбора
   - В нефинальных сценах может быть 1 или {CHOICES_AVG} выбора
   - Провалы (failX) могут появляться на любом этапе
   - Финали (finalX) только в конце веток
3. Пример:
   1. start_scene: 1 -> scene1, 2 -> scene2
   2. scene1: 1 -> scene3
   3. scene2: 1 -> fail1, 2 -> scene4
   4. scene3: 1 -> final1, 2 -> fail2
   5. scene4: 1 -> scene5
   6. scene5: 1 -> fail3, 2 -> final2
   7. scene6: 1 -> final3
   8. scene7: 1 -> fail4
"""
print(prompt_structure)
response_structure = ollama.chat(
    model=MODEL_TO_USE,
    messages=messages_strict + [{"role": "user", "content": prompt_structure}],
    options={"temperature":0.3}
)
structure = response_structure["message"]["content"]
print(structure)


Сгенерируй структуру квеста по следующим правилам:
1. Формат строго: 
   <номер>. <scene_id>: <Выбор1> -> <target1>[, <Выбор2> -> <target2>]
2. Условия:
   - Минимум 8 уникальных сцен (не считая финалов/провалов)
   - В стартовой сцене 3 выбора
   - В нефинальных сценах может быть 1 или 2 выбора
   - Провалы (failX) могут появляться на любом этапе
   - Финали (finalX) только в конце веток
3. Пример:
   1. start_scene: 1 -> scene1, 2 -> scene2
   2. scene1: 1 -> scene3
   3. scene2: 1 -> fail1, 2 -> scene4
   4. scene3: 1 -> final1, 2 -> fail2
   5. scene4: 1 -> scene5
   6. scene5: 1 -> fail3, 2 -> final2
   7. scene6: 1 -> final3
   8. scene7: 1 -> fail4

1. start_scene: 1 -> scene1, 2 -> scene2, 3 -> scene3
2. scene1: 1 -> scene5, 2 -> fail1
3. scene2: 1 -> scene6, 2 -> final1
4. scene3: 1 -> scene7, 2 -> scene8
5. scene4: 1 -> fail2, 2 -> scene9
6. scene5: 1 -> scene10, 2 -> fail3
7. scene6: 1 -> fail4, 2 -> final2
8. scene7: 1 -> final3, 2 -> fail5
9. scene8: 1 -> fail6, 2 -> fina

In [31]:
extracted_info = {
    "setting": "Киберпанк",
    "main_charachter": "Хакер-одиночка",
    "goal": "взломать замок на двери, найти и забрать чип с вирусом"
}


json_strict = f"""
Преврати структуру квеста в JSON строго по этим правилам:
1. Сеттинг: {extracted_info["setting"]}
2. Главный герой: {extracted_info["main_charachter"]}
3. Финальная цель: {extracted_info["goal"]}"""+"""
Правила генерации:
- Сохрани ВСЕ оригинальные scene_id и связи между сценами
- JSON состоит из последовательного описания сцен (формат ниже)
- Для каждой сцены добавь:
  - "text": описание ситуации (2-3 предложения)
  - "choices": массив выборов (минимум 1, максимум 3)
- Требования к выборам:
  50% - абсурдные/смешные варианты
  50% - логичные, учитывающие предыдущие события
-Каждый выбор - конкретное действие (глагол + дополнение)
-"after": немедленный результат выбора (1-2 предложения)
- Провалы (failX) должны быть нелепыми, но в рамках сеттинга
- Финалы (finalX) должны соотноситься с целью: """ + extracted_info["goal"] + """
Формат строго JSON, где сцены выводятся через запятую, без комментариев. Пример формата одной сцены:
{
  "scene_id": "village_gate",
  "text": "Вы подходите к воротам деревни. Страж в броне пристально смотрит на вас.",
  "choices": [
    {
    "text": "Поговорить со стражем",
    "after": "Вы начинаете заикаться от вида мускулов стража",
    "next_scene_id": "guard_dialogue"
    },
    {
    "text": "Пойти обратно в лес",
    "after": "Вы подскользнулись на банановой кожуре и сломали ногу",
    "next_scene_id": "broken_leg"
    }
    ]
}
"""
messages_strict_json = [{
        "role": "system",
        "content": json_strict}]
response_json = ollama.chat(
    model=MODEL_TO_USE,
    options = {"temperature": 0.5},
    messages=messages_strict_json + [{"role": "user", "content": structure}],
    format = "json"
)

print(response_json["message"]["content"])

{
    "start_scene": "scene1",
    "scenes": [
        {
            "scene_id": "scene1",
            "text": "Вы оказались у двери, заблокированной сложным замком. На вашем плече висит рюкзак с инструментами и шприцем для взлома.",
            "choices": [
                {"text": "Начать взламывать замок", 
                 "after": "Вы достаете мультитул, но на вас нападает разъяренный кот из-за угла. Он кусается за ногу и сбегает с вашей сумкой.",
                 "next_scene_id": "fail1"
                },
                {"text": "Нажать кнопку вызова помощника",
                 "after": "Из двери выходит робот, который говорит: \"Ты что-то хочешь?\"",
                 "next_scene_id": "robot_dialogue"
                }
            ]
        },
        {
            "scene_id": "fail1",
            "text": "Ваше нога кровоточит. Вам нужно найти аптечку или какое-то другое место для лечения.",
            "choices": [
                {"text": "Попробовать сделать стимулятор из ш

In [33]:
import json
import re

quest_json = response_json["message"]["content"]
# Извлечение JSON (на случай, если модель "добавила мусор")
quest_json = json.loads(
    re.search(r'\{.*\}', quest_json, flags=re.DOTALL).group(0)
)

print(quest_json)  # Готовый JSON для работы

{'start_scene': 'scene1', 'scenes': [{'scene_id': 'scene1', 'text': 'Вы оказались у двери, заблокированной сложным замком. На вашем плече висит рюкзак с инструментами и шприцем для взлома.', 'choices': [{'text': 'Начать взламывать замок', 'after': 'Вы достаете мультитул, но на вас нападает разъяренный кот из-за угла. Он кусается за ногу и сбегает с вашей сумкой.', 'next_scene_id': 'fail1'}, {'text': 'Нажать кнопку вызова помощника', 'after': 'Из двери выходит робот, который говорит: "Ты что-то хочешь?"', 'next_scene_id': 'robot_dialogue'}]}, {'scene_id': 'fail1', 'text': 'Ваше нога кровоточит. Вам нужно найти аптечку или какое-то другое место для лечения.', 'choices': [{'text': 'Попробовать сделать стимулятор из шприца и остатков лекарств в рюкзаке', 'after': 'Вы делаете себе укол, но вместо улучшения самочувствия начинаете петь песню про кота. Ваша нога не перестала кровоточить.', 'next_scene_id': 'scene1'}, {'text': 'Убегать в лес', 'after': 'Вы замечаете, что вместо леса перед вами 

In [36]:
# Запись строки в файл
with open("quest.json", "w", encoding="utf-8") as file:
    json.dump(quest_json, file, ensure_ascii=False, indent=4)